# 2026/02/23 BME1462: Morphological Image Processing II 

## Topics of Focus: 
 
### 9.4 Hit-or-Miss Transform 

### 9.5 Morphological Algorithms 

Material covered based on Digital Image Processing, Gonzalez and Woods, Prentice Hall, 4th edition 

Images Acquired from the Following Sources: 

### Recap: Morphological Image Processing I 

Recall the principles of mathematical morphology 